In [1]:
!pip install annoy
!pip install implicit

In [4]:
import annoy
import numpy as np
import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise import SVD, evaluate, accuracy
from surprise import dump
from surprise.model_selection import cross_validate, train_test_split
from surprise.model_selection import GridSearchCV 

In [5]:
from surprise import NMF
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNBaseline
from surprise import KNNWithZScore
from surprise import SlopeOne
from surprise import SVDpp
from surprise import NormalPredictor
from surprise import BaselineOnly
from surprise import CoClustering

In [6]:
from surprise import get_dataset_dir
import io

In [7]:
from tqdm import tqdm_notebook as tqdm

In [70]:
import implicit

In [33]:
df = pd.read_csv('item_item_test.csv')
df.head(30)

,Unnamed: 0,userId,movieId,rating,timestamp
0,90,4,181,0.5,1127946789
1,296,4,1760,0.5,1114050751
2,484,4,3593,1.0,1115782046
3,1631,19,181,2.0,836433987
4,2655,38,4125,1.0,982853654
5,4450,54,810,3.0,868126354
6,5189,56,4255,3.5,1381372980
7,6455,72,90890,3.5,1459980875
8,6506,72,125916,0.5,1459979959
9,7119,75,125916,2.5,1511888779


In [34]:
df.drop(['Unnamed: 0','timestamp'], inplace=True,axis=1)

In [35]:
df.head()

,userId,movieId,rating
0,4,181,0.5
1,4,1760,0.5
2,4,3593,1.0
3,19,181,2.0
4,38,4125,1.0


In [36]:
len(df)

63163

In [61]:
def rec_gridsearch(model,param_grid,cv):
    gs = GridSearchCV(model, param_grid, measures=['rmse', 'mae'], cv=cv, n_jobs=-1)
    gs.fit(data)
    # best RMSE score
    print(gs.best_score['rmse'])

    # combination of parameters that gave the best RMSE score
    print(gs.best_params['rmse'])

## Lets narrow down candidates.    

In [51]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=10, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse') 

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matr

,test_rmse,fit_time,test_time
Algorithm,,,
KNNBaseline,1.020727,89.183144,7.468731
SVDpp,1.040596,14.320231,0.210154
KNNBasic,1.053583,83.150241,6.483075
KNNWithMeans,1.059923,89.239505,6.966381
NMF,1.060838,6.355041,0.069143
KNNWithZScore,1.062559,93.642434,7.276902
SlopeOne,1.064206,0.392056,0.145317
SVD,1.065767,4.401206,0.075216
CoClustering,1.081905,4.598551,0.051666


## SVD

In [144]:
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(df,reader)

In [40]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    1.0480  1.0639  1.0572  1.0631  1.0695  1.0577  1.0587  1.0828  1.0779  1.0816  1.0660  0.0110  
MAE (testset)     0.8468  0.8610  0.8527  0.8584  0.8670  0.8529  0.8580  0.8763  0.8746  0.8800  0.8628  0.0107  
Fit time          4.23    4.24    4.17    4.20    4.20    4.22    4.29    4.22    4.29    4.16    4.22    0.04    
Test time         0.06    0.06    0.06    0.06    0.06    0.07    0.06    0.06    0.14    0.06    0.07    0.03    


{'test_rmse': array([1.04799704, 1.06393136, 1.05718242, 1.0631157 , 1.06948358,
        1.0576798 , 1.05870252, 1.08278422, 1.07792866, 1.0815916 ]),
 'test_mae': array([0.84680658, 0.86103314, 0.85271596, 0.85842769, 0.86704902,
        0.85287666, 0.85803869, 0.87626983, 0.87460099, 0.88001262]),
 'fit_time': (4.225666761398315,
  4.244113922119141,
  4.16943097114563,
  4.201759099960327,
  4.200515985488892,
  4.22236704826355,
  4.287075757980347,
  4.219311237335205,
  4.285580635070801,
  4.1620237827301025),
 'test_time': (0.05500006675720215,
  0.05860090255737305,
  0.057804107666015625,
  0.05599403381347656,
  0.05787920951843262,
  0.07165002822875977,
  0.05598306655883789,
  0.0560460090637207,
  0.1419200897216797,
  0.06035280227661133)}

In [42]:
param_grid = {'n_epochs': [10, 20, 50], 'n_factors': [1,5,10],'lr_all': [0.001, 0.005, 0.01,0.05],
              'reg_all': [0.4, 0.6], 'biased': [True, False],}
rec_gridsearch(SVD,param_grid,3)

1.0465191432486414
{'n_epochs': 50, 'n_factors': 10, 'lr_all': 0.01, 'reg_all': 0.4, 'biased': True}


In [53]:
param_grid = {'n_epochs': [50,75,100], 'n_factors': [5,10,20],'lr_all': [0.01,0.05],
              'reg_all': [0.3,0.4,0.5], 'biased': [True, False],}
rec_gridsearch(SVD,param_grid,3)

1.0424537695080438
{'n_epochs': 75, 'n_factors': 20, 'lr_all': 0.01, 'reg_all': 0.3, 'biased': True}


In [64]:
param_grid = {'n_epochs': [75], 'n_factors': [20,30,40],'lr_all': [0.01],
              'reg_all': [0.1,0.2,0.3], 'biased': [True],}
rec_gridsearch(SVD,param_grid,3)

1.0298727259675857
{'n_epochs': 75, 'n_factors': 40, 'lr_all': 0.01, 'reg_all': 0.1, 'biased': True}


In [65]:
param_grid = {'n_epochs': [75], 'n_factors': [40,50,60],'lr_all': [0.01],
              'reg_all': [0.01,0.05,0.1], 'biased': [True],}
rec_gridsearch(SVD,param_grid,3)

1.0282990610139882
{'n_epochs': 75, 'n_factors': 60, 'lr_all': 0.01, 'reg_all': 0.1, 'biased': True}


In [66]:
param_grid = {'n_epochs': [75], 'n_factors': [60,70,80,90],'lr_all': [0.01],
              'reg_all': [0.1], 'biased': [True],}
rec_gridsearch(SVD,param_grid,3)

1.0281911365332073
{'n_epochs': 75, 'n_factors': 90, 'lr_all': 0.01, 'reg_all': 0.1, 'biased': True}


Marginal improvement at this point. Moving on to other models.

## NMF

In [37]:
nmf = NMF()
cross_validate(nmf, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    1.0633  1.0634  1.0668  1.0630  1.0625  1.0599  1.0577  1.0702  1.0516  1.0759  1.0634  0.0063  
MAE (testset)     0.7956  0.7947  0.7967  0.8000  0.8004  0.7915  0.7877  0.7993  0.7878  0.8044  0.7958  0.0053  
Fit time          7.55    7.07    7.60    7.27    7.57    7.09    7.65    7.90    7.39    7.00    7.41    0.28    
Test time         0.07    0.06    0.06    0.07    0.06    0.05    0.06    0.09    0.08    0.15    0.08    0.03    


{'test_rmse': array([1.06328433, 1.06336099, 1.06677292, 1.06301131, 1.06252209,
        1.05991413, 1.05772675, 1.07020364, 1.05158049, 1.07592665]),
 'test_mae': array([0.79562841, 0.79471426, 0.7966913 , 0.79999204, 0.80043335,
        0.79146986, 0.78771769, 0.79933589, 0.78783922, 0.80444307]),
 'fit_time': (7.5549468994140625,
  7.073349952697754,
  7.60212516784668,
  7.266840934753418,
  7.566035985946655,
  7.087186813354492,
  7.645038843154907,
  7.904263973236084,
  7.385869979858398,
  7.002677917480469),
 'test_time': (0.07328200340270996,
  0.05996417999267578,
  0.06066727638244629,
  0.06553912162780762,
  0.059427738189697266,
  0.054814815521240234,
  0.06008100509643555,
  0.08710718154907227,
  0.0838470458984375,
  0.1524968147277832)}

In [39]:
# Not tuning LR/Reg until biased is determined to be true or not. 
param_grid = {'n_epochs': [10, 20, 50], 'n_factors': [1,5,10], 'biased': [True, False],}
tqdm(rec_gridsearch(NMF,param_grid,9))

1.057280091186513
{'n_epochs': 50, 'n_factors': 1, 'biased': False}


0it [00:00, ?it/s]

### KNN Baseline

In [78]:
knn = KNNBaseline()

In [79]:
cross_validate(knn, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matr

{'test_rmse': array([1.02117399, 1.01472005, 1.01711235, 1.00567749, 1.01832867,
        1.01538854, 1.02541524, 1.03927229, 1.01674642, 1.02055621]),
 'test_mae': array([0.78972274, 0.79656566, 0.79622076, 0.78623501, 0.79834623,
        0.79404881, 0.80474513, 0.81302319, 0.79518576, 0.79507834]),
 'fit_time': (81.97987985610962,
  85.09711527824402,
  83.33498191833496,
  82.5072431564331,
  81.26806092262268,
  85.20557284355164,
  85.35262894630432,
  82.57301998138428,
  82.91954588890076,
  80.7125871181488),
 'test_time': (6.315085172653198,
  6.4196648597717285,
  7.319612741470337,
  6.577861309051514,
  6.937384843826294,
  6.732741832733154,
  6.29314398765564,
  6.4974470138549805,
  7.580524206161499,
  6.67586612701416)}

In [83]:
param_grid_knn = {'k': [20,30,50,60]}
knnbaseline_gs = GridSearchCV(KNNBaseline, param_grid_knn, measures=['rmse', 'mae'], cv=4, n_jobs=-1)
knnbaseline_gs.fit(data)

In [162]:
print(knnbaseline_gs.best_params)
print(knnbaseline_gs.best_score)

{'rmse': {'k': 35}, 'mae': {'k': 30}}
{'rmse': 1.0327849906778375, 'mae': 0.8090094874055599}


In [86]:
param_grid_knn = {'k': [30,35,40,45]}
knnbaseline_gs = GridSearchCV(KNNBaseline, param_grid_knn, measures=['rmse', 'mae'], cv=4, n_jobs=-1)
knnbaseline_gs.fit(data)
print(knnbaseline_gs.best_params)
print(knnbaseline_gs.best_score)

{'rmse': {'k': 35}, 'mae': {'k': 30}}
{'rmse': 1.0327849906778375, 'mae': 0.8090094874055599}


### SVDpp

In [54]:
svdpp = SVDpp()
cross_validate(svdpp, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    1.0417  1.0315  1.0367  1.0386  1.0307  1.0364  1.0249  1.0468  1.0484  1.0428  1.0379  0.0070  
MAE (testset)     0.8153  0.8088  0.8099  0.8155  0.7991  0.8077  0.8022  0.8201  0.8223  0.8163  0.8117  0.0071  
Fit time          13.81   13.53   13.70   14.03   13.13   12.79   12.96   12.73   12.63   12.64   13.19   0.50    
Test time         0.31    0.15    0.18    0.16    0.14    0.15    0.14    0.15    0.14    0.15    0.17    0.05    


{'test_rmse': array([1.04172288, 1.03148326, 1.0367124 , 1.03858046, 1.03072872,
        1.03638924, 1.02494009, 1.04676632, 1.04843384, 1.04284449]),
 'test_mae': array([0.81527772, 0.80879415, 0.80990328, 0.8154974 , 0.79912627,
        0.80766324, 0.80224503, 0.82013032, 0.82229712, 0.81628458]),
 'fit_time': (13.805808782577515,
  13.52705192565918,
  13.697068929672241,
  14.026280879974365,
  13.12869930267334,
  12.78980016708374,
  12.958942174911499,
  12.731549978256226,
  12.634726285934448,
  12.638183832168579),
 'test_time': (0.31453514099121094,
  0.14533495903015137,
  0.18267416954040527,
  0.15613913536071777,
  0.14376091957092285,
  0.1474299430847168,
  0.1381058692932129,
  0.15408015251159668,
  0.1385819911956787,
  0.15295100212097168)}

In [62]:
param_grid_svdpp = {'n_epochs': [50,75,100], 'n_factors': [5,10,20],'lr_all': [0.01,0.05],
              'reg_all': [0.3,0.4,0.5],'random_state':[7]}
rec_gridsearch(SVDpp,param_grid_svdpp,7)

1.031005864411734
{'n_epochs': 75, 'n_factors': 20, 'lr_all': 0.01, 'reg_all': 0.3, 'random_state': 7}


In [67]:
param_grid_svdpp = {'n_epochs': [75], 'n_factors': [20,30,40],'lr_all': [0.01],
              'reg_all': [0.1,0.2,0.3],'random_state':[7]}
rec_gridsearch(SVDpp,param_grid_svdpp,7)

1.013783707348949
{'n_epochs': 75, 'n_factors': 40, 'lr_all': 0.01, 'reg_all': 0.1, 'random_state': 7}


In [68]:
param_grid_svdpp = {'n_epochs': [75], 'n_factors': [40,50,60],'lr_all': [0.01],
              'reg_all': [0.05,0.1],'random_state':[7]}
rec_gridsearch(SVDpp,param_grid_svdpp,7)

1.012264700140572
{'n_epochs': 75, 'n_factors': 50, 'lr_all': 0.01, 'reg_all': 0.1, 'random_state': 7}


Winner!

### Annoy

In [87]:
from annoy import AnnoyIndex

In [90]:
ls

335k.csv                    average/
615k.csv                    first_ml_dataset.csv
Data_Gathering.ipynb        good_bad_movie_list.csv
EDA.ipynb                   goodbad.db
Flask Implementation.ipynb  gtag_test.csv
README.md                   item_item_test.csv
Recommender_Modeling.ipynb  ml-latest/
SQL Import.ipynb            ward/


In [93]:
annoy_df = pd.read_csv('good_bad_movie_list.csv')
annoy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 32 columns):
Unnamed: 0               128 non-null int64
index                    128 non-null int64
movieId                  128 non-null int64
title                    128 non-null object
genres                   128 non-null object
imdbId                   128 non-null object
tmdbId                   126 non-null float64
('count', 0.5)           128 non-null float64
('count', 1.0)           128 non-null float64
('count', 1.5)           128 non-null float64
('count', 2.0)           128 non-null float64
('count', 2.5)           128 non-null float64
('count', 3.0)           128 non-null float64
('count', 3.5)           128 non-null float64
('count', 4.0)           128 non-null float64
('count', 4.5)           128 non-null float64
('count', 5.0)           128 non-null float64
Cheesy                   128 non-null float64
Cult                     128 non-null float64
Cult Classic             128 n

### Final Model

In [145]:
train, test = train_test_split(data, test_size=.2)

In [146]:
final_model = SVDpp(n_epochs=75, n_factors=50, 
              lr_all=0.01, reg_all=0.1,random_state=7)

In [151]:
final_model.fit(train)
predictions = final_model.test(test)
accuracy.rmse(predictions)

RMSE: 1.0159


1.0158505817425685

In [148]:
accuracy.mae(predictions)

MAE:  0.5167


0.5167495186189889

In [129]:
dump.dump('final_model', algo=final_model, predictions=predictions, verbose=1)

The dump has been saved as file final_model


In [131]:
preds, model = dump.load('final_model')

In [154]:
predictions

[Prediction(uid=60900, iid=3573, r_ui=4.0, est=1.8148834648064411, details={'was_impossible': False}),
 Prediction(uid=47265, iid=3799, r_ui=5.0, est=4.762999747372759, details={'was_impossible': False}),
 Prediction(uid=281535, iid=1760, r_ui=3.0, est=1.7667679050762473, details={'was_impossible': False}),
 Prediction(uid=245339, iid=90890, r_ui=0.5, est=0.8113782021349213, details={'was_impossible': False}),
 Prediction(uid=259920, iid=2383, r_ui=0.5, est=1.4232502708429215, details={'was_impossible': False}),
 Prediction(uid=121261, iid=3593, r_ui=3.0, est=1.756846889092994, details={'was_impossible': False}),
 Prediction(uid=202682, iid=125916, r_ui=3.5, est=1.8861287289483741, details={'was_impossible': False}),
 Prediction(uid=124637, iid=1323, r_ui=0.5, est=1.480585390847311, details={'was_impossible': False}),
 Prediction(uid=140598, iid=4241, r_ui=1.0, est=1.5454718346606553, details={'was_impossible': False}),
 Prediction(uid=207864, iid=181, r_ui=2.0, est=1.9341919946510697,

In [149]:
sim_options = {'name': 'pearson_baseline', 'user_based': False}
algo = KNNBaseline(k=40,sim_options=sim_options)
algo.fit(train)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [150]:
knn_predictions = algo.test(test)
accuracy.rmse(knn_predictions)

RMSE: 1.0517


1.051715769725699

In [155]:
algo1 = KNNBaseline()

In [156]:
algo1.fit(train)
knn1_predictions = algo1.test(test)
accuracy.rmse(knn1_predictions)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0255


1.0254548940449109

In [160]:
algo2 = KNNBaseline(k=30)

In [161]:
algo2.fit(train)
knn2_predictions = algo2.test(test)
accuracy.rmse(knn2_predictions)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0252


1.025218431688024

In [183]:
algo3 = KNNBaseline(k=29,sim_options=sim_options)

In [184]:
algo3.fit(train)
knn3_predictions = algo3.test(test)
accuracy.rmse(knn3_predictions)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0550


1.0549952693931341

In [185]:
sim_options = {'name': 'msd', 'user_based': True}
algo4 = KNNBaseline(k=5,sim_options=sim_options)

In [186]:
algo4.fit(train)
knn4_predictions = algo4.test(test)
accuracy.rmse(knn4_predictions)

Estimating biases using als...
Computing the msd similarity matrix...


KeyboardInterrupt: 

In [187]:
ls

335k.csv                    final_model
615k.csv                    first_ml_dataset.csv
Data_Gathering.ipynb        good_bad_movie_list.csv
EDA.ipynb                   goodbad.db
Implementation.ipynb        gtag_test.csv
README.md                   item_item_final.csv
Recommender_Modeling.ipynb  item_item_test.csv
SQL Import.ipynb            ml-latest/
average/                    ward/


## Redoing everything with a new dataset

In [8]:
df2 = pd.read_csv('item_item_final.csv')
df2.head(30)

,Unnamed: 0,userId,movieId,rating,timestamp
0,90,4,181,0.5,1127946789
1,296,4,1760,0.5,1114050751
2,341,4,2152,0.5,1115781964
3,417,4,2816,2.5,1123989912
4,466,4,3268,0.5,1114050659
5,484,4,3593,1.0,1115782046
6,757,4,42721,0.5,1145938348
7,1631,19,181,2.0,836433987
8,1657,19,286,3.0,849710700
9,1749,19,591,3.0,849710721


In [9]:
df2.drop(['Unnamed: 0','timestamp'], inplace=True,axis=1)

In [10]:
df2.head()

,userId,movieId,rating
0,4,181,0.5
1,4,1760,0.5
2,4,2152,0.5
3,4,2816,2.5
4,4,3268,0.5


In [12]:
reader = Reader(rating_scale=(0.5, 5))
data2 = Dataset.load_from_df(df2,reader)

In [13]:
train2, test2 = train_test_split(data2, test_size=.2)

### KNN Baseline

In [26]:
sim_options = {'name': 'msd',
               'min_support': 5,
               'user_based': False}
base1 = KNNBaseline(k=30,sim_options=sim_options)

In [21]:
base1.fit(train2)
base1_preds = base1.test(test2)
accuracy.rmse(base1_preds)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0197


1.0196939903010263

In [22]:
sim_options = {'name': 'cosine',
               'min_support': 5,
               'user_based': False}
base2 = KNNBaseline(k=30,sim_options=sim_options)

In [23]:
base2.fit(train2)
base2_preds = base2.test(test2)
accuracy.rmse(base2_preds)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0241


1.024058366731553

In [27]:
sim_options = {'name': 'pearson_baseline',
               'min_support': 5,
               'user_based': False}
base3 = KNNBaseline(k=30,sim_options=sim_options)

In [28]:
base3.fit(train2)
base3_preds = base3.test(test2)
accuracy.rmse(base3_preds)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.0098


1.0098460059651595

In [31]:
sim_options = {'name': 'pearson_baseline',
               'min_support': 5,
               'user_based': False}
base4 = KNNBaseline(k=35,sim_options=sim_options)

In [32]:
base4.fit(train2)
base4_preds = base4.test(test2)
accuracy.rmse(base4_preds)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.0098


1.0098252215921415

In [33]:
sim_options = {'name': 'pearson_baseline',
               'min_support': 5,
               'user_based': False}
base5 = KNNBaseline(k=40,sim_options=sim_options)

In [34]:
base5.fit(train2)
base5_preds = base5.test(test2)
accuracy.rmse(base5_preds)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.0098


1.0098465914432893

In [49]:
sim_options = {'name': 'pearson_baseline',
               'min_support': 5,
               'user_based': False}
base6 = KNNBaseline(k=39,sim_options=sim_options)

In [50]:
base6.fit(train2)
base6_preds = base6.test(test2)
accuracy.rmse(base6_preds)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.0098


1.0098372002372284

In [51]:
sim_options = {'name': 'pearson_baseline',
               'min_support': 5,
               'user_based': False}
base7 = KNNBaseline(k=37,sim_options=sim_options)

In [52]:
base7.fit(train2)
base7_preds = base7.test(test2)
accuracy.rmse(base7_preds)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.0098


1.0098314020112866

In [53]:
sim_options = {'name': 'pearson_baseline',
               'min_support': 5,
               'user_based': False}
base8 = KNNBaseline(k=36,sim_options=sim_options)

In [54]:
base8.fit(train2)
base8_preds = base8.test(test2)
accuracy.rmse(base8_preds)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.0098


1.0098283820135368

In [55]:
sim_options = {'name': 'pearson_baseline',
               'min_support': 5,
               'user_based': False}
base9 = KNNBaseline(k=33,sim_options=sim_options)

In [56]:
base9.fit(train2)
base9_preds = base9.test(test2)
accuracy.rmse(base9_preds)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.0098


1.0098364010834906

In [57]:
sim_options = {'name': 'pearson_baseline',
               'min_support': 5,
               'user_based': False}
base10 = KNNBaseline(k=10,sim_options=sim_options)

In [58]:
base10.fit(train2)
base10_preds = base10.test(test2)
accuracy.rmse(base10_preds)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.0099


1.0098848285440225

In [59]:
sim_options = {'name': 'pearson_baseline',
               'min_support': 5,
               'user_based': False}
base11 = KNNBaseline(k=20,sim_options=sim_options)

In [60]:
base11.fit(train2)
base11_preds = base11.test(test2)
accuracy.rmse(base11_preds)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.0098


1.0098036979411174

In [61]:
sim_options = {'name': 'pearson_baseline',
               'min_support': 5,
               'user_based': False}
base12 = KNNBaseline(k=25,sim_options=sim_options)

In [62]:
base12.fit(train2)
base12_preds = base12.test(test2)
accuracy.rmse(base12_preds)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.0099


1.0098544168866566

In [63]:
sim_options = {'name': 'pearson_baseline',
               'min_support': 5,
               'user_based': False}
base13 = KNNBaseline(k=21,sim_options=sim_options)

In [64]:
base13.fit(train2)
base13_preds = base13.test(test2)
accuracy.rmse(base13_preds)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.0098


1.009794033164408

### BEST MODEL ^^^^^^^^

In [67]:
sim_options = {'name': 'pearson_baseline',
               'min_support': 5,
               'user_based': False}
base14 = KNNBaseline(k=22,sim_options=sim_options)

In [68]:
base14.fit(train2)
base14_preds = base14.test(test2)
accuracy.rmse(base14_preds)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.0098


1.009816116980724

### KNNMeans

In [38]:
sim_options = {'name': 'msd',
               'min_support': 5,
               'user_based': False}
means1 = KNNWithMeans(k=30,sim_options=sim_options)

In [36]:
means1.fit(train2)
means1_preds = means1.test(test2)
accuracy.rmse(means1_preds)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0294


1.0294320120270934

In [39]:
sim_options = {'name': 'cosine',
               'min_support': 5,
               'user_based': False}
means2 = KNNWithMeans(k=30,sim_options=sim_options)

In [40]:
means2.fit(train2)
means2_preds = means2.test(test2)
accuracy.rmse(means2_preds)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0337


1.0337170543942928

In [41]:
sim_options = {'name': 'pearson_baseline',
               'min_support': 5,
               'user_based': False}
means3 = KNNWithMeans(k=30,sim_options=sim_options)

In [42]:
means3.fit(train2)
means3_preds = means3.test(test2)
accuracy.rmse(means3_preds)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.0203


1.0203067696333434

### KNN Basic

In [43]:
sim_options = {'name': 'msd',
               'min_support': 5,
               'user_based': False}
basic1 = KNNBasic(k=30,sim_options=sim_options)

In [44]:
basic1.fit(train2)
basic1_preds = basic1.test(test2)
accuracy.rmse(basic1_preds)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0450


1.045005286085737

In [45]:
sim_options = {'name': 'cosine',
               'min_support': 5,
               'user_based': False}
basic2 = KNNBasic(k=30,sim_options=sim_options)

In [46]:
basic2.fit(train2)
basic2_preds = basic2.test(test2)
accuracy.rmse(basic2_preds)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0508


1.0508391483421877

In [47]:
sim_options = {'name': 'pearson_baseline',
               'min_support': 5,
               'user_based': False}
basic3 = KNNBasic(k=30,sim_options=sim_options)

In [48]:
basic3.fit(train2)
basic3_preds = basic3.test(test2)
accuracy.rmse(basic3_preds)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.0352


1.035231088637645